**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.**

In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install deap

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from IPython.display import Image, display
from deap import creator, base, tools, algorithms, gp
import matplotlib.pyplot as plt
import math, random, operator
import numpy as np
import pydot

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
!mkdir -p output

def plot_syntax_tree(gp, ind, show=True):
    nodes, edges, labels = gp.graph(ind)
    graph = pydot.graph_from_edges(edges)

    for node in nodes:
        graph.add_node(pydot.Node(node, label=labels[node]))

    if show:
        img = Image(graph.create_png())
        display(img)

    return graph

# Symbolic Regression Using Genetic Programming

Our next example will illustrate how another method – genetic programming – can be used to perform symbolic regression. As the name suggests, symbolic regression is a type of regression, where the resulting regressor is an analytic expression (unlike a neural network, for an instance, where the regressor is implicit and not easily interpretable).

## Generating a Synthetic Dataset

We'll start by generating a synthetic dataset – we will take points from a polynomial and add Gaussian noise to them.

In [ ]:
X = np.arange(-1, 1, 0.1)
Y = X**4 + X**3 + X**2 + X + np.random.normal(0, 0.05, size=X.shape)

We'll visualize the resulting data:

In [ ]:
plt.figure()
plt.scatter(X, Y)
plt.grid(ls='--')
plt.xlabel("x")
plt.ylabel("y")

Our fitness function will now compute the mean squared error, which the genetic program makes on our samples:

In [ ]:
def evaluateGP(individual, X, Y):
    func = toolbox.compile(expr=individual)
    sqerrors = [(func(x) - y)**2 for x, y in zip(X, Y)]
    return math.fsum(sqerrors) / len(X), # the fitness can be multidimensional;
                                         # we return it as an tuple, hence the comma

## Terminals and Non-Terminals

We will next create a set of terminals and non-terminals that GP will be able to select from. We also specify that our function will have a single argument.

In [ ]:
pset = gp.PrimitiveSet("main", 1)

Arguments can be named to make the results more readable – here we are naming the 0th argument 'x'.

In [ ]:
pset.renameArguments(ARG0='x')

We can use arbitrary Python functions as terminals. The number specifies the arity of the function, i.e. how many arguments it has. We are adding an ephemeral constant from $\{-1, 0, 1\}$ as a terminal.

In [ ]:
# A workaround to avoid exceptions when running the code repeatedly.
try: del gp.randConstant
except: None

pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
# integer constants
pset.addEphemeralConstant("randConstant", lambda: random.randint(-1, 1))
# real-valued constants
# pset.addEphemeralConstant("randConstant", lambda: random.uniform(-1, 1))

In [ ]:
# Aby sme sa zbavili výnimky pri opakovanom spustení skriptu.
try: del gp.randConstant
except: None

pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
# celočíselné konštanty
pset.addEphemeralConstant("randConstant", lambda: random.randint(-1, 1))
# reálnočíselné konštanty
# pset.addEphemeralConstant("randConstant", lambda: random.uniform(-1, 1))

We can also add our own functions to the set:

In [ ]:
def foo(x):
    return x*2 + x

pset.addPrimitive(foo, 1)

## Setting Up the Parameters

We specify that the goal is to minimize the objective function. Our individuals will be represented by syntax trees (GP).

In [ ]:
# We register that fitness is to be minimized: hence the -1 weight.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# We register the type of the individual; it is going to be based
# on type gp.PrimitiveTree, i.e. a syntax tree. Its fitness
# is to be minimized (we specify function FitnessMin, which
# we have defined above).
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

We define how our population is to be generated, evaluated, etc.

In [ ]:
# We create a toolbox of basic operators that GP will use.
toolbox = base.Toolbox()
# We register the tree-generating method.
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
# We register the method that creates individuals.
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
# We register the method that creates the population.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# We register the function that compiles the syntax tree.
toolbox.register("compile", gp.compile, pset=pset)
# We use our evaluation function to evaluate fitness.
toolbox.register("evaluate", evaluateGP, X=X, Y=Y)

We define the genetic operators and the selection method.

In [ ]:
# We choose one-point crossover for GP (subtree swap) as our crossover method.
toolbox.register("mate", gp.cxOnePoint)
# When applying mutation, we will generate full trees.
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
# We register the mutation method (replacing random subtrees by newly generated ones).
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
# We use tournament selection with tournament size of 3 as our selection method.
toolbox.register("select", tools.selTournament, tournsize=3)

We will display some statics while running the algorithm: the average, minimal and maximal fitness, etc. 

In [ ]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

We are going to create an archive that will hold the best individual.

In [ ]:
hof = tools.HallOfFame(1)

## Running the Algorithm

We will generate the initial population and then run the algorithm:

In [ ]:
pop = toolbox.population(n=300)
final_pop = algorithms.eaSimple(pop, toolbox, cxpb=0.5,
                                mutpb=0.1, ngen=40,
                                stats=mstats, halloffame=hof)

## Evaluating the Results

We display the resulting individual and its fitness:

In [ ]:
best_ind = hof[0]

print("Individual: {}\nfitness = {}".format(
    best_ind, best_ind.fitness.values[0]))

We can also visualize and inspect the syntax tree corresponding to our regressor.

In [ ]:
plot_syntax_tree(gp, best_ind);

### The Regression Curve

Finally, we can also plot our original data and the resulting regression curve.

In [ ]:
func = toolbox.compile(expr=best_ind)

plt.figure(figsize=(8, 6))
plt.plot(X, Y, 'bx')
plt.plot(X, [func(x) for x in X], 'r')
plt.grid()
plt.xlabel("x")
plt.ylabel("y")

plt.legend(["regressor", "original data"])